In [118]:
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/13.TrafficPrediction')
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from scipy import stats

In [37]:
df_train=pd.read_csv('train_aWnotuB.csv',parse_dates=['DateTime'])
df_test=pd.read_csv('test_BdBKkAj.csv',parse_dates=['DateTime'])


In [14]:
df_train.head()

,DateTime,Junction,Vehicles,ID
0,2015-11-01 00:00:00,1,15,20151101001
1,2015-11-01 01:00:00,1,13,20151101011
2,2015-11-01 02:00:00,1,10,20151101021
3,2015-11-01 03:00:00,1,7,20151101031
4,2015-11-01 04:00:00,1,9,20151101041


In [136]:
def get_vars(df):
    df['year']=df['DateTime'].dt.year
    df['month']=df['DateTime'].dt.month
    df['day']=df['DateTime'].dt.day
    df['hour']=df['DateTime'].dt.hour
    df['day_of_week']=df['DateTime'].dt.dayofweek
    df['sunday_flag']=(df['day_of_week']==6).astype(np.int64)
    df['saturday_flag']=(df['day_of_week']==5).astype(np.int64)
    df['weekday_flag']=(df['day_of_week']<=4).astype(np.int64)
    df['encoded_month']=12*df['year']+df['month']-24191
    df['encoded_day']=(df['DateTime']-date(2015,11,1)).dt.days
    return df

train=get_vars(df_train)
test=get_vars(df_test)

In [137]:
#Clean the day level traffic
day_junction_avg=train.groupby(['Junction','encoded_day','encoded_month'])['Vehicles'].mean().reset_index()
day_junction_avg_prev=day_junction_avg.drop('encoded_month',axis=1)
day_junction_avg_prev['encoded_day']=day_junction_avg_prev['encoded_day']+1
day_junction_avg_next=day_junction_avg.drop('encoded_month',axis=1)
day_junction_avg_next['encoded_day']=day_junction_avg_next['encoded_day']-1
day_junction_avg_1=pd.merge(day_junction_avg_next,day_junction_avg_prev,on=['encoded_day','Junction'],how='outer')
day_junction_avg_1=pd.merge(day_junction_avg,day_junction_avg_1,on=['encoded_day','Junction'],how='outer')
day_junction_avg_1['%Change1']=100.0*(day_junction_avg_1['Vehicles_y']/day_junction_avg_1['Vehicles']-1)
day_junction_avg_1['%Change2']=100.0*(day_junction_avg_1['Vehicles']/day_junction_avg_1['Vehicles_x']-1)
day_junction_avg_1['%Change']=(day_junction_avg_1['%Change1']*day_junction_avg_1['%Change2']).fillna(0)
day_junction_avg_1['%Change'].describe(percentiles=[0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.95,0.99])
day_junction_avg_1=day_junction_avg_1[np.logical_and(day_junction_avg_1['%Change']<300,day_junction_avg_1['%Change']>-300)]

In [165]:
#Get Monthly averages and coefficients for regression
#slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
regression_params={}
monthly_avgs=day_junction_avg_1.groupby(['encoded_month','Junction'])['Vehicles'].mean().reset_index()
for name, group in monthly_avgs.groupby(['Junction']):
    group=group[group['Vehicles']>7]
    slope, intercept, r_value, p_value, std_err = stats.linregress(group['encoded_month'],group['Vehicles'])
    regression_params[name]=[round(slope,2), round(intercept,2)]

print regression_params

{1: [2.93, 18.59], 2: [0.9, 6.18], 3: [0.5, 8.65], 4: [0.17, 5.19]}


In [153]:
#Score using monthly averages
def score_monthly_regression(df):
    slope=df['Junction'].map(lambda x: regression_params[x][0])
    intercept=df['Junction'].map(lambda x: regression_params[x][1])
    return intercept+slope*df['encoded_month']

test['Vehicles']=score_monthly_regression(test)
test[['ID','Vehicles']].to_csv('Submission_Monthly_Regression.csv',index=False)

In [163]:
train['Vehicles_monthly_regression']=score_monthly_regression(train)
test['Vehicles_monthly_regression']=score_monthly_regression(test)

train['Delta']=train['Vehicles']-train['Vehicles_monthly_regression']
weekday_delta=train.groupby(['day_of_week','Junction'])['Delta'].mean().to_dict()
train['Vehicles_monthly_regression']=train['Vehicles_monthly_regression']+\
                        train.apply(lambda x: weekday_delta[(x['day_of_week'],x['Junction'])],axis=1)
    
test['Vehicles']=test['Vehicles_monthly_regression']+\
                        test.apply(lambda x: weekday_delta[(x['day_of_week'],x['Junction'])],axis=1) 

test[['ID','Vehicles']].to_csv('Submission_Monthly_Regression_with_weekday.csv',index=False)

In [161]:
train['Delta']=train['Vehicles']-train['Vehicles_monthly_regression']
train.groupby(['day_of_week','Junction'])['Delta'].mean().unstack()

Junction,1,2,3,4
day_of_week,,,,
0,1.082148e-15,-9.130202e-15,5.226981e-15,-2.728587e-15
1,7.384472e-15,-2.075819e-15,7.728173e-15,1.730809e-15
2,-2.058124e-14,6.615398e-15,-5.274623e-16,9.735802e-16
3,3.811340e-15,-3.396176e-15,-5.529846e-15,2.180592e-15
4,-5.934801e-15,1.330566e-15,3.661609e-15,-1.113070e-14
5,-1.427695e-13,-1.012110e-14,1.182172e-14,-1.115552e-14
6,-3.044308e-14,4.396653e-15,-5.842921e-15,6.023672e-15
